<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [19]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [20]:
raw_data[:10]

'First Citi'

In [21]:
dataset[:10]

['First Citizen:\n',
 'Before we proceed any further, hear me speak.\n',
 '\n',
 'All:\n',
 'Speak, speak.\n',
 '\n',
 'First Citizen:\n',
 'You are all resolved rather to die than to famish?\n',
 '\n',
 'All:\n']

In [23]:
vocab = set(raw_data)

In [24]:
vocab_len = len(vocab)

In [25]:
vocab_len

65

In [26]:
stoi = {char:i for i, char in enumerate(vocab)}

In [27]:
itos = {i:char for i, char in enumerate(vocab)}

In [39]:
def get_example():
  random_index = math.floor(random.random() * (len(raw_data) - 4 ))
  example = raw_data[random_index:random_index+3]
  for ch1, ch2, ch3 in zip(example, example[1:], example[2:]):
    x = torch.tensor([stoi[ch1], stoi[ch2]])
    y = torch.tensor([stoi[ch3]])
  return x, y

get_example()

(tensor([36, 10]), tensor([21]))

In [31]:
import torch

In [50]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([130, 65])

In [33]:
import torch.nn.functional as functional

In [41]:
example = get_example()

In [47]:
xenc = functional.one_hot(example[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([ 0.8143, -1.8154,  0.0081,  2.6326,  0.5033,  1.8110, -0.8996,  0.2832,
         1.9693,  0.9668, -0.1522,  0.8453, -1.5940, -0.0572, -3.0750,  1.2177,
        -3.3580,  2.7750,  0.0064, -0.2187, -0.9074, -1.5966, -0.4230,  1.3723,
         0.9080, -1.8304,  1.3765,  1.4351,  1.9158, -1.1289, -1.2883,  1.6175,
         3.6898, -1.1279,  1.1523, -2.5885, -2.7998, -0.2776, -0.6877, -0.4831,
         0.5950, -3.2997,  1.4111, -0.6673, -2.6603, -2.3671,  1.1314, -0.7100,
         1.1258,  1.3411,  1.2518, -2.9245, -0.6943,  0.5719,  0.1754, -0.7823,
         0.6075, -0.9650, -0.2062,  0.0074,  2.6850,  0.4327, -0.2739, -0.3603,
        -1.1846])

In [57]:
num_epochs = 100
examples_per_epoch = 1000
lr = 0.01

In [58]:
for epoch in range(num_epochs):
  for example in range(examples_per_epoch):
    xs, ys = get_example()
    ys = functional.one_hot(ys, num_classes=65)
    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten()
    logits = xenc @ ws
    probs = logits.exp()
    loss = ((ys - probs) ** 2).mean()
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

Streaming output truncated to the last 5000 lines.
0.516082227230072
0.13042473793029785
0.08761241286993027
0.4764077067375183
0.5319918394088745
0.44644924998283386
0.22146964073181152
0.09954598546028137
0.3279951810836792
0.11079815030097961
0.12294498085975647
0.540583074092865
0.3829449415206909
0.06735190004110336
0.21732006967067719
0.299565851688385
0.13047176599502563
0.36808574199676514
0.2978001534938812
0.24615024030208588
0.08589738607406616
0.03576020151376724
0.11189985275268555
0.030138833448290825
0.07406529039144516
0.32663026452064514
0.10369308292865753
0.0899607241153717
0.30985134840011597
0.2531582713127136
0.061430830508470535
0.10028813779354095
0.38330748677253723
0.29422998428344727
0.11515657603740692
1.1579066514968872
0.13379120826721191
0.057588931173086166
0.11467953771352768
0.031195776537060738
0.033434461802244186
0.6701489686965942
0.24812372028827667
0.5708349347114563
0.6656742691993713
0.4681115448474884
0.2610607445240021
0.1013897955417633
0.44

In [64]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=65).float().flatten()
  logits = xenc @ ws
  probs = logits.exp()
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([stoi[last_two[0]], stoi[last_two[1]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

"RomFnJDFnCtipOpFehOz:ucXJvXb?js-RRkPFqPXvbIwa:OxvSOnPnOSPs&:VXL 3xm!cE?jv-ntXIPcv&:JM,:ijbDMMc:thzaxRMzasCbx:qj-ON!yiqeCFMFvFvVSqVubBwTR-EnOYqsOFTWuDwxn!HbbwmirxGQeSNEei?AH'vCj:,3wek. !z&.:S?-:rXxiP-Pq:aPJhObPR;wn-iFLAEpwbD!h!TnEFY!Lw;KM&ubkUzsFSnyWuzzxnGemMixze!FiyAehYwSMhMx-nsexZnWP\nRSejTQCzqPpFMJAG!J?MsQqFiJ?hTijHS:SGEFg:pqMMVNQjC:mAwYXuFeq$nqgtMY!luzZ3$IkNN!FpxPDz Cb'qsuEjPq?Dn-QdwvX:cmKAtQyC$SPeqTUfAVTOtqTFPTvx:y\nDsawXOeB;3PRqhJkJhet\nbCAMoi!RiM?JPs.:ewNibChum-whqey\n$ow\nNF qwbPhWwbLm:zr; RgPe,E"